In [1]:
import pandas as pd 
import numpy as np 
from numpy import dot 
from sklearn.metrics.pairwise import cosine_similarity 
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer 
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, cross_validate

In [2]:
pd.set_option('display.max_columns', 500)

https://www.datasciencelearner.com/sklearn-cosine-similarity-implementation/

In [3]:
df = pd.read_csv('../data/final_df.csv', index_col=0)

In [4]:
df

,track_id,track_name,track_popularity,duration_ms,explicit,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artist_id,followers,genres,artist_name,artist_popularity
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,1922-02-22,0.645,0.44500,0,-13.338,1,0.4510,0.674,0.744000,0.1510,0.127,104.851,3,45tIt06XoI0Iio4LBEVpls,91.0,NaN,Uli,4
1,0PH9AACae1f957JAavhOl2,Lazy Boi,0,157333,0,1922-02-22,0.298,0.46000,1,-18.645,1,0.4530,0.521,0.856000,0.4360,0.402,87.921,4,45tIt06XoI0Iio4LBEVpls,91.0,NaN,Uli,4
2,2SiNuAZ6jIU9xhClRKXcST,Sketch,0,87040,0,1922-02-22,0.634,0.00399,5,-29.973,0,0.0377,0.926,0.919000,0.1050,0.396,79.895,4,45tIt06XoI0Iio4LBEVpls,91.0,NaN,Uli,4
3,4vV7uBcF2AnjNTOejBS5oL,L'enfer,0,40000,0,1922-02-22,0.657,0.32500,10,-14.319,0,0.2540,0.199,0.856000,0.0931,0.105,81.944,5,45tIt06XoI0Iio4LBEVpls,91.0,NaN,Uli,4
4,598LlBn6jpEpVbLjmZPsYV,Graphite,0,104400,0,1922-02-22,0.644,0.68400,7,-8.247,1,0.1990,0.144,0.802000,0.0847,0.138,100.031,4,45tIt06XoI0Iio4LBEVpls,91.0,NaN,Uli,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
470033,0MmaEacabpK8Yp3Mdeo5uY,下雨天,50,265846,0,2020-02-25,0.528,0.67300,4,-3.639,1,0.0314,0.143,0.000000,0.0989,0.297,130.066,4,5VGgFE9nPgMfEnYiPT5J2B,929.0,chinese viral pop,芝麻,36
470034,1dKxf4Ht2SsKLyXfSDJAgy,The Cutest Puppy,67,82500,0,2020-10-30,0.609,0.01720,8,-28.573,1,0.1180,0.996,0.973000,0.1080,0.890,68.619,4,7vgGpuiXdNlCmc994PlMlz,23.0,instrumental lullaby,Laureen Conrad,52
470035,0SjsIzJkZfDU7wlcdklEFR,John Brown's Song,66,185250,0,2020-03-20,0.562,0.03310,1,-25.551,1,0.1030,0.996,0.961000,0.1110,0.386,63.696,3,4MxqhahGRT4BPz1PilXGeu,91.0,instrumental lullaby,Gregory Oberle,55
470036,5rgu12WBIHQtvej2MdHSH0,云与海,50,258267,0,2020-09-26,0.560,0.51800,0,-7.471,0,0.0292,0.785,0.000000,0.0648,0.211,131.896,4,1QLBXKM5GCpyQQSVMNZqrZ,896.0,chinese viral pop,阿YueYue,38


In [5]:
df = df.dropna(subset=['genres'])

In [6]:
df.isna().sum()

track_id             0
track_name           0
track_popularity     0
duration_ms          0
explicit             0
release_date         0
danceability         0
energy               0
key                  0
loudness             0
mode                 0
speechiness          0
acousticness         0
instrumentalness     0
liveness             0
valence              0
tempo                0
time_signature       0
artist_id            0
followers            0
genres               0
artist_name          0
artist_popularity    0
dtype: int64

In [7]:
num_cols = ['danceability', 'energy', 'loudness', 'key', 'mode',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'time_signature', 'track_popularity']

In [8]:
df['track_id']

56        07A5yehtSnoedViJAZkNnc
57        08FmqUhxtyLTn6pAh6bk45
58        0JV4iqw2lSKJaHBQZ0e5zK
59        0l3BQsVJ7F76wlN5QhJzaP
60        0xJCJ9XSNcdTIz0QKmhtEn
                   ...          
470033    0MmaEacabpK8Yp3Mdeo5uY
470034    1dKxf4Ht2SsKLyXfSDJAgy
470035    0SjsIzJkZfDU7wlcdklEFR
470036    5rgu12WBIHQtvej2MdHSH0
470037    0NuWgxEp51CutD2pJoF4OM
Name: track_id, Length: 432228, dtype: object

In [9]:
scaler = StandardScaler()

scaler.fit(df[num_cols])

StandardScaler()

In [10]:
scaled_data = pd.DataFrame(scaler.fit_transform(df[num_cols]), columns=num_cols)

In [11]:
scaled_data

,danceability,energy,loudness,key,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_popularity
0,-0.823764,-1.595823,-2.533133,-1.203130,0.712639,-0.265077,1.719770,-0.279239,-0.010532,-0.419442,0.367961,2.512874,-1.733620
1,-1.534985,-1.936469,-4.034035,0.501498,0.712639,-0.269892,1.722753,3.581239,-0.590591,-0.656883,1.703360,-1.992675,-1.733620
2,-0.106249,-2.165909,-2.986436,0.217393,0.712639,0.347572,1.725736,3.645853,-0.354271,0.193948,-1.482568,-1.992675,-1.733620
3,-1.226580,-0.789683,-1.638480,-0.919025,0.712639,-0.052636,1.692923,-0.370211,3.840421,0.751936,1.355317,0.260100,-1.733620
4,-1.421693,-1.931508,-3.808933,0.785602,0.712639,-0.254245,1.719770,3.800927,-0.488544,-1.361293,-1.121934,-1.992675,-1.733620
...,...,...,...,...,...,...,...,...,...,...,...,...,...
432223,-0.232129,0.454667,1.349378,-0.350816,0.712639,-0.384237,-0.818768,-0.373145,-0.617983,-1.052619,0.356079,0.260100,1.190996
432224,0.277684,-2.256445,-4.169494,0.785602,0.712639,0.136936,1.725736,3.818157,-0.569108,1.294093,-1.718040,0.260100,2.185366
432225,-0.018133,-2.190713,-3.500607,-1.203130,0.712639,0.046664,1.725736,3.766466,-0.552995,-0.700414,-1.884214,-1.992675,2.126874
432226,-0.030721,-0.186111,0.501206,-1.487234,-1.403235,-0.397477,1.096322,-0.373145,-0.801132,-1.392951,0.417850,0.260100,1.190996


In [12]:
df_track_info = df[['track_id', 'track_name', 'artist_name']]
df_track_info

,track_id,track_name,artist_name
56,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,Ignacio Corsini
57,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,Ignacio Corsini
58,0JV4iqw2lSKJaHBQZ0e5zK,Martín Fierro - Remasterizado,Ignacio Corsini
59,0l3BQsVJ7F76wlN5QhJzaP,El Vendaval - Remasterizado,Ignacio Corsini
60,0xJCJ9XSNcdTIz0QKmhtEn,La Maleva - Remasterizado,Ignacio Corsini
...,...,...,...
470033,0MmaEacabpK8Yp3Mdeo5uY,下雨天,芝麻
470034,1dKxf4Ht2SsKLyXfSDJAgy,The Cutest Puppy,Laureen Conrad
470035,0SjsIzJkZfDU7wlcdklEFR,John Brown's Song,Gregory Oberle
470036,5rgu12WBIHQtvej2MdHSH0,云与海,阿YueYue


In [13]:
df_track_info.head()

,track_id,track_name,artist_name
56,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,Ignacio Corsini
57,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,Ignacio Corsini
58,0JV4iqw2lSKJaHBQZ0e5zK,Martín Fierro - Remasterizado,Ignacio Corsini
59,0l3BQsVJ7F76wlN5QhJzaP,El Vendaval - Remasterizado,Ignacio Corsini
60,0xJCJ9XSNcdTIz0QKmhtEn,La Maleva - Remasterizado,Ignacio Corsini


In [33]:
df.isna().sum()

track_id             0
track_name           0
track_popularity     0
duration_ms          0
explicit             0
release_date         0
danceability         0
energy               0
key                  0
loudness             0
mode                 0
speechiness          0
acousticness         0
instrumentalness     0
liveness             0
valence              0
tempo                0
time_signature       0
artist_id            0
followers            0
genres               0
artist_name          0
artist_popularity    0
dtype: int64

In [14]:
df.head()

,track_id,track_name,track_popularity,duration_ms,explicit,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artist_id,followers,genres,artist_name,artist_popularity
56,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,1922-03-21,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.021800,0.212,0.457,130.418,5,5LiOoJbxVSAMkBS2fUm3X2,3528.0,"tango, vintage tango",Ignacio Corsini,23
57,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,1922-03-21,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.918000,0.104,0.397,169.980,3,5LiOoJbxVSAMkBS2fUm3X2,3528.0,"tango, vintage tango",Ignacio Corsini,23
58,0JV4iqw2lSKJaHBQZ0e5zK,Martín Fierro - Remasterizado,0,181173,0,1922-03-29,0.548,0.0391,6,-23.228,1,0.1530,0.996,0.933000,0.148,0.612,75.595,3,5LiOoJbxVSAMkBS2fUm3X2,3528.0,"tango, vintage tango",Ignacio Corsini,23
59,0l3BQsVJ7F76wlN5QhJzaP,El Vendaval - Remasterizado,0,153533,0,1922-03-21,0.370,0.3720,2,-17.138,1,0.0865,0.985,0.000681,0.929,0.753,159.669,4,5LiOoJbxVSAMkBS2fUm3X2,3528.0,"tango, vintage tango",Ignacio Corsini,23
60,0xJCJ9XSNcdTIz0QKmhtEn,La Maleva - Remasterizado,0,181440,0,1922-03-29,0.339,0.0958,8,-26.944,1,0.0530,0.994,0.969000,0.123,0.219,86.279,3,5LiOoJbxVSAMkBS2fUm3X2,3528.0,"tango, vintage tango",Ignacio Corsini,23


In [34]:
df_track_info.join(scaled_data).isna().sum()

track_id                0
track_name              0
artist_name             0
danceability        30810
energy              30810
loudness            30810
key                 30810
mode                30810
speechiness         30810
acousticness        30810
instrumentalness    30810
liveness            30810
valence             30810
tempo               30810
time_signature      30810
track_popularity    30810
dtype: int64

The following code is from 
https://stackoverflow.com/questions/36538780/merging-dataframes-on-index-with-pandas

In [35]:
df_mark = df_track_info.join(scaled_data, how='inner')

In [38]:
# refers to the columns to the right of artist name (all num cols)
df_mark.iloc[:,3:]

,danceability,energy,loudness,key,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_popularity
56,-1.446869,-1.930681,-3.926021,1.637916,-1.403235,-0.041803,1.725736,2.926482,-0.300562,0.379944,0.933857,2.512874,-1.733620
57,-1.062936,-1.310573,-2.526493,0.217393,0.712639,-0.233181,1.707838,3.473548,-0.262965,0.415560,-1.879927,0.260100,-1.733620
58,-1.723805,-1.695040,-2.734995,-0.066712,0.712639,-0.220543,1.722753,-0.292162,-0.144805,0.003995,1.945178,0.260100,-1.733620
59,-0.792294,-1.674370,-2.487980,0.217393,-1.403235,-0.132678,1.704855,3.340013,-0.563737,0.724234,1.656644,0.260100,-1.733620
60,-1.465751,-1.178284,-2.248712,0.785602,0.712639,-0.123049,1.698889,3.775081,-0.198514,-0.063280,2.026054,-1.992675,-1.733620
...,...,...,...,...,...,...,...,...,...,...,...,...,...
432223,-0.232129,0.454667,1.349378,-0.350816,0.712639,-0.384237,-0.818768,-0.373145,-0.617983,-1.052619,0.356079,0.260100,1.190996
432224,0.277684,-2.256445,-4.169494,0.785602,0.712639,0.136936,1.725736,3.818157,-0.569108,1.294093,-1.718040,0.260100,2.185366
432225,-0.018133,-2.190713,-3.500607,-1.203130,0.712639,0.046664,1.725736,3.766466,-0.552995,-0.700414,-1.884214,-1.992675,2.126874
432226,-0.030721,-0.186111,0.501206,-1.487234,-1.403235,-0.397477,1.096322,-0.373145,-0.801132,-1.392951,0.417850,0.260100,1.190996


In [41]:
# looks at just the first three track data columns
df_mark.iloc[:,:3]

,track_id,track_name,artist_name
56,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,Ignacio Corsini
57,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,Ignacio Corsini
58,0JV4iqw2lSKJaHBQZ0e5zK,Martín Fierro - Remasterizado,Ignacio Corsini
59,0l3BQsVJ7F76wlN5QhJzaP,El Vendaval - Remasterizado,Ignacio Corsini
60,0xJCJ9XSNcdTIz0QKmhtEn,La Maleva - Remasterizado,Ignacio Corsini
...,...,...,...
432223,65o7zOY79D5vqOJJNm1l3T,下雨的晚上,Dadado Huang
432224,7D9yBn5ivJUao1v4jmVdgG,25歲,Dadado Huang
432225,6di4lDxW9XThds6gIHVRtL,跟你出去玩,Dadado Huang
432226,4EoOSTT7iBjHxSfOfmB8Iq,香格里拉,Dadado Huang


In [56]:
# From a track_name to a track_id (Track must exactly match)
track = 'House of the Rising Sun'
df_mark[df_mark['track_name'] == track][['artist_name', 'track_id']]

,artist_name,track_id
39924,Pete Seeger,2bT8S1aNa6nAcMrDVfw3Xr
74229,The Animals,4mn2kNTqiGLwaUR8JdhJ1l
74234,The Animals,3XC7Jd6SfrQYKZJ6inyRHK
74238,The Animals,1D8MLfoyeJr3W7sEYsDoyv
74306,The Animals,0ev91hkSfD8mR15VBiAHts
119396,Idris Muhammad,0ggo2PQvf6xG7RWF5z1pHN
261078,Five Finger Death Punch,2SgQsZIWs0UC01ibNOBu7q
397123,Gildran,0jIDNggEsTbXOr8h8WBtTa


In [57]:
def find_track_id(track):
    return df_mark[df_mark['track_name'] == track][['artist_name', 'track_id']]

In [58]:
find_track_id("House of the Rising Sun")

,artist_name,track_id
39924,Pete Seeger,2bT8S1aNa6nAcMrDVfw3Xr
74229,The Animals,4mn2kNTqiGLwaUR8JdhJ1l
74234,The Animals,3XC7Jd6SfrQYKZJ6inyRHK
74238,The Animals,1D8MLfoyeJr3W7sEYsDoyv
74306,The Animals,0ev91hkSfD8mR15VBiAHts
119396,Idris Muhammad,0ggo2PQvf6xG7RWF5z1pHN
261078,Five Finger Death Punch,2SgQsZIWs0UC01ibNOBu7q
397123,Gildran,0jIDNggEsTbXOr8h8WBtTa


In [63]:
trackID = '2bT8S1aNa6nAcMrDVfw3Xr'

def cosSim(trackID):
    '''
    Takes in a trackID,
    returns top 10 most similar tracks
    '''
    track_num_cols = df_mark[df_mark['track_id'] == trackID].iloc[:,3:]
    cos_sim = cosine_similarity
    return 
    
    #return track_name

In [80]:
cosSim(trackID).values

array([[ 0.98261127, -1.03359195, -0.88814028, -0.9190253 , -1.4032348 ,
        -0.18262854,  1.18282917, -0.37313833, -0.268336  ,  0.60155611,
         0.13610016,  0.26009989,  0.1966268 ]])

In [85]:
song_0 = df[num_cols][df['track_id'] == '2bT8S1aNa6nAcMrDVfw3Xr']
song_1 = df_mark.iloc[:,3:][df_mark['track_id'] == '1gqkRc9WtOpnGIqxf2Hvzr']

In [86]:
song_1

,danceability,energy,loudness,key,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_popularity
249853,-0.584592,-0.607785,0.266808,-0.919025,0.712639,-0.391459,1.275302,-0.373145,-0.714123,-0.922026,-0.349291,0.2601,1.190996


In [65]:
track_num_cols = df_mark[df_mark['track_id'] == trackID].iloc[:,3:]

In [79]:
for index, row in df_mark.iloc[:,3:][:5].iterrows():
    print(row)#cosine_similarity(track_num_cols, row)
    
# [row for index, row in df_mark.iterrows()]

danceability       -1.446869
energy             -1.930681
loudness           -3.926021
key                 1.637916
mode               -1.403235
speechiness        -0.041803
acousticness        1.725736
instrumentalness    2.926482
liveness           -0.300562
valence             0.379944
tempo               0.933857
time_signature      2.512874
track_popularity   -1.733620
Name: 56, dtype: float64
danceability       -1.062936
energy             -1.310573
loudness           -2.526493
key                 0.217393
mode                0.712639
speechiness        -0.233181
acousticness        1.707838
instrumentalness    3.473548
liveness           -0.262965
valence             0.415560
tempo              -1.879927
time_signature      0.260100
track_popularity   -1.733620
Name: 57, dtype: float64
danceability       -1.723805
energy             -1.695040
loudness           -2.734995
key                -0.066712
mode                0.712639
speechiness        -0.220543
acousticness        1.

In [94]:
song_1

,danceability,energy,loudness,key,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_popularity
249853,-0.584592,-0.607785,0.266808,-0.919025,0.712639,-0.391459,1.275302,-0.373145,-0.714123,-0.922026,-0.349291,0.2601,1.190996


In [77]:
cos_sim = cosine_similarity(song_0, song_1)
cos_sim

array([[0.9305183]])

In [98]:
num_cols

['danceability',
 'energy',
 'loudness',
 'key',
 'mode',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo',
 'time_signature',
 'track_popularity']

In [102]:
song_0_sc = scaler.transform(song_0)

In [100]:
df_mark.head()[num_cols]

,danceability,energy,loudness,key,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_popularity
56,-1.446869,-1.930681,-3.926021,1.637916,-1.403235,-0.041803,1.725736,2.926482,-0.300562,0.379944,0.933857,2.512874,-1.73362
57,-1.062936,-1.310573,-2.526493,0.217393,0.712639,-0.233181,1.707838,3.473548,-0.262965,0.415560,-1.879927,0.260100,-1.73362
58,-1.723805,-1.695040,-2.734995,-0.066712,0.712639,-0.220543,1.722753,-0.292162,-0.144805,0.003995,1.945178,0.260100,-1.73362
59,-0.792294,-1.674370,-2.487980,0.217393,-1.403235,-0.132678,1.704855,3.340013,-0.563737,0.724234,1.656644,0.260100,-1.73362
60,-1.465751,-1.178284,-2.248712,0.785602,0.712639,-0.123049,1.698889,3.775081,-0.198514,-0.063280,2.026054,-1.992675,-1.73362


In [128]:
df_mark['artist_name'].unique()

array(['Ignacio Corsini', 'Dick Haymes', 'Mistinguett', ...,
       'Green River Ordinance', 'Koala Liu', 'Dadado Huang'], dtype=object)

In [129]:
df_mark.loc[df_mark['artist_name'] == 'Ignacio Corsini']

,track_id,track_name,artist_name,danceability,energy,loudness,key,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_popularity
56,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,Ignacio Corsini,-1.446869,-1.930681,-3.926021,1.637916,-1.403235,-0.041803,1.725736,2.926482,-0.300562,0.379944,0.933857,2.512874,-1.733620
57,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,Ignacio Corsini,-1.062936,-1.310573,-2.526493,0.217393,0.712639,-0.233181,1.707838,3.473548,-0.262965,0.415560,-1.879927,0.260100,-1.733620
58,0JV4iqw2lSKJaHBQZ0e5zK,Martín Fierro - Remasterizado,Ignacio Corsini,-1.723805,-1.695040,-2.734995,-0.066712,0.712639,-0.220543,1.722753,-0.292162,-0.144805,0.003995,1.945178,0.260100,-1.733620
59,0l3BQsVJ7F76wlN5QhJzaP,El Vendaval - Remasterizado,Ignacio Corsini,-0.792294,-1.674370,-2.487980,0.217393,-1.403235,-0.132678,1.704855,3.340013,-0.563737,0.724234,1.656644,0.260100,-1.733620
60,0xJCJ9XSNcdTIz0QKmhtEn,La Maleva - Remasterizado,Ignacio Corsini,-1.465751,-1.178284,-2.248712,0.785602,0.712639,-0.123049,1.698889,3.775081,-0.198514,-0.063280,2.026054,-1.992675,-1.733620
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611,7z8Vqn6pqLbdJrJOs7ht0r,Caminito de la Sierra - Remasterizado,Ignacio Corsini,-1.604219,-0.913704,-0.224787,-1.487234,-1.403235,-0.323454,1.612382,-0.363539,0.843445,-1.788687,-1.249256,2.512874,-1.558143
612,7k3NpVXmmR8BPPElS1bwbJ,La Quema - Remasterizado,Ignacio Corsini,-1.906331,-1.798392,-2.226136,-1.487234,0.712639,-0.327065,1.719770,2.267418,-0.526140,-1.662052,-1.378537,2.512874,-1.558143
613,7m41BoE7089tdvt3tadI0U,A la Luz de un Candil - Remasterizado,Ignacio Corsini,-2.221030,-1.310573,-1.340336,-1.487234,0.712639,-0.276512,1.710821,3.288321,1.101250,-1.024917,-1.397000,-6.498223,-1.499651
614,7vGkgN23wKgIR41gcFbWVX,Gauchito Lindo - Remasterizado,Ignacio Corsini,-1.251755,-1.517276,-1.222805,0.501498,-1.403235,-0.234385,1.651161,1.965886,-0.488544,-1.167382,-0.247622,-1.992675,-0.856235


In [119]:
similarity = {}
for t_id in df_mark['track_id'].head():
    song_1 = df_mark[num_cols][df_mark['track_id'] == t_id]
    sim = cosine_similarity(song_0_sc, song_1)[0][0]
    similarity[t_id] = sim

In [124]:
list(similarity.values())[0]

0.5038497689969865

In [123]:
df_mark.loc[df_mark['track_id'] == list(similarity.keys())[0]]

,track_id,track_name,artist_name,danceability,energy,loudness,key,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_popularity
56,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,Ignacio Corsini,-1.446869,-1.930681,-3.926021,1.637916,-1.403235,-0.041803,1.725736,2.926482,-0.300562,0.379944,0.933857,2.512874,-1.73362
